<h2 style="text-align: center;"> Modelo de Regresión</h2>

### Autor: Francisco Lackington


## Contexto
- Trabajaremos una base de datos sobre los precios de las viviendas en Boston, utilizada en el paper Harrison Jr, D., & Rubinfeld, D. L. (1978). Hedonic housing prices and the demand for clean air. Journal of environmental economics and management, 5(1), 81-102.


- Nuestro objetivo es desarrollar un modelo predictivo para el valor mediano de las casas mediante el entrenamiento de un modelo de regresión lineal.

    - `crim`: Tasa de criminalidad por sector de Boston
    - `zn`: proporción de terreno residencial asignado para terrenos baldíos.
    - `indus`: proporción de negocios no asociados al comercio por sector.
    - `chas`: Dummy. 1 si el sector colinda con el río Charles, 0 de lo contrario.
    - `nox`: Concentración de dióxido de carbono
    - `rm`: cantidad promedio de habitaciones por casa.
    - `age`: proporción de casas construídas antes de 1940
    - `dis`: distancia promedio a cinco centros de empleos.
    - `rad`: índice de accesibilidad a autopistas.
    - `tax`: nivel de impuestos asociados a viviendas.
    - `ptratio`: razón alumno:profesor por sector de Boston.
    - `black`: proporción de afroamericanos por sector de Boston.
    - `lstat`: porcentaje de población de estratos bajos.
    - `medv`: valor mediano de las casas en \$1.000s

In [1]:
# Importe librerías y módulos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Módulos de sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
# Importando Data set y eliminando columna 'Unnamed: 0'

df = pd.read_csv('boston.csv').drop(columns=['Unnamed: 0'])
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
# Generemos un analisis descriptivo inicial

df.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


- Se observa que la tasa de criminalidad maxima es de 88.9 puntos, sin. embargo el 50% de los registros presenta una tasa en torno a 0.25 puntos.
- Se observa que en promedio las casa tienen 6 habitaciones y las mayores pueden llegar a 9
- Se observa que en promedio hay un 12% de población de estratos bajos

## Machine Leaarning

In [4]:
# Generamos la particion del data set

X = df.copy()
y = X.pop('medv')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 123)

Ahora implementaremos dos versiones del modelo lineal:
    - Con intercepto y atributos normalizados.
    - Sin intercepto y atributos no normalizados.

In [5]:
# Generando los dos modelos

modelo_1 = LinearRegression(normalize=True).fit(X_train, y_train)
modelo_2 = LinearRegression(fit_intercept=False).fit(X_train, y_train)

In [6]:
# Almacenando la prediccion para ambos modelos

y_hat_1 = modelo_1.predict(X_test)
y_hat_2 = modelo_2.predict(X_test)

## Obtención de métricas


In [7]:
# Definir función
def report_scores(yhat, ytest):
    '''
    Esta funcion imprime las metricas de Error Cuadratico Medio y R2 de un vector de datos predichos vs 
    el vector de datos de prueba
    
    Input
        - yhat: vector de datos predichos por el modelo
        - ytest: vector de datos de test
    Output
        - Metricas de MSE y R2
    
    '''
    mse_modelo = mean_squared_error(ytest, yhat).round(2)
    r2_modelo = r2_score(ytest, yhat).round(3)
    
    print(f'MSE: {mse_modelo}\nr2: {r2_modelo}')

In [8]:
# Veamos las metricas para el primer modelo
report_scores(y_hat_1, y_test)

MSE: 29.95
r2: 0.652


In [9]:
# Veamos metricas para segundo modelo
report_scores(y_hat_2, y_test)

MSE: 34.26
r2: 0.602


**¿Cuál de los 2 es el mejor modelo? ¿Por qué?**

- R: Entre los dos modelos es mejor el modelo 1, dado que el Error Cuadrático Medio es menor y además el r2 es mayor. Recordar que en estos casos de compración de modelos se busca minimizar el MSE

## Refactorización del modelo

In [10]:
# Definir función
def fetch_features(data, vector = 'medv'):
    # genero correlacion de atributos con vector objetivo
    corr_vec = data.corr()[vector]
    # veo el valor absoluto para las correlaciones obtenidas
    abs_corr_vec = abs(corr_vec)
    # ordenando los valores obtenidos
    sort_abs_corr_vec = abs_corr_vec.sort_values(ascending = False)
    # eliminando el vector objetivo
    sort_abs_corr_vec.pop(vector)
    sort_abs_corr_vec
    
    return sort_abs_corr_vec
    

In [11]:
# Utilizando la funcion generada

fetch_features(df)

lstat      0.737663
rm         0.695360
ptratio    0.507787
indus      0.483725
tax        0.468536
nox        0.427321
crim       0.388305
rad        0.381626
age        0.376955
zn         0.360445
black      0.333461
dis        0.249929
chas       0.175260
Name: medv, dtype: float64

## Refactorización del modelo predictivo

In [12]:
# seleccionando los 6 atributos con mayor correlacion
variables = fetch_features(df)[:6]
variables

lstat      0.737663
rm         0.695360
ptratio    0.507787
indus      0.483725
tax        0.468536
nox        0.427321
Name: medv, dtype: float64

In [13]:
# generando una lista para los atributos con mayor correlacion
lista = list(variables.index)
lista


['lstat', 'rm', 'ptratio', 'indus', 'tax', 'nox']

In [14]:
# Ejecutando segmentación

X_red = df.loc[:, lista]
y = df['medv']

# como ocupo misma semilla, no es necesario volver a definir las variables y_train, y_test
X_train, X_test, _, _ = train_test_split(X_red, y, test_size = 0.33, random_state = 123)

In [15]:
# Genere un "modelo 3" usando los hiperparámetros del mejor modelo anterior, para este caso normalize = True
# Entrenando modelo con fit()

modelo_3 = LinearRegression(normalize=True).fit(X_train, y_train)

In [16]:
# Almacenando las predicciones del modelo entrenado
y_hat_3 = modelo_3.predict(X_test)

In [17]:
# Llame a la función del ejercicio 4 para reportar las métricas del nuevo modelo
report_scores(y_hat_3, y_test)

MSE: 37.24
r2: 0.567


**¿Es mejor este nuevo modelo (depurado, solo con atributos más relevantes) que el mejor de los 2 modelos generados en primera instancia (saturados, con todos los atributos)?**

- R: En esta caso el modelo empeoro, dado que el MSE aumentó y se quiere minimizar dicho valor

## Predicción de casos
- A continuación se generaron dos arrays que representan el peor escenario posible ( worst_neighbor ) y el mejor escenario posible ( best_neighbor ).
- Ingrese los arrays en el tercer modelo entrenado y reporte cuál sería el valor esperado dada las condiciones.

In [18]:
# Escenarios a predecir
worst_neighbor = np.array([37.9, 12.6, 3.5, 27.7, 187, 0.87]).reshape(1, -1)
best_neighbor = np.array([1.73, 22, 8.7, 0.46, 711, 0.38]).reshape(1, -1)

In [19]:
# Predicción para peor vecino
modelo_3.predict(worst_neighbor)

array([59.55548606])

In [20]:
# Preficción para mejor vecino
modelo_3.predict(best_neighbor)

array([120.75805057])

**¿En cuál de los 2 casos se predice un mayor valor de la vivienda?**

- R: el caso que predice un mayor valor para la vivienda es el best_neighbor como se esperaba